In [8]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
from datetime import datetime
import pandas as pd


class ForexSystem:
    def __init__(self, *args, **kwargs):
        self.api = API(access_token=kwargs["access_token"])
        self.instrument = None
        self.account_id = None
        self.qty = 0
        self.resample_interval = '10s'
        self.mean_period_short = 5
        self.mean_period_long = 20
        self.buy_threshold = 1.0
        self.sell_threshold = 1.0

        self.prices = pd.DataFrame()
        self.beta = 0
        self.is_position_opened = False
        self.opening_price = 0
        self.executed_price = 0
        self.unrealized_pnl = 0
        self.realized_pnl = 0
        self.position = 0
        self.dt_format = "%Y-%m-%dT%H:%M:%S.%fZ"
        
    def begin(self, **params):
        self.instrument = params["instruments"]
        self.account_id = params["accountId"]
        self.qty = params["qty"]
        self.resample_interval = params["resample_interval"]
        self.mean_period_short = params["mean_period_short"]
        self.mean_period_long = params["mean_period_long"]
        self.buy_threshold = params["buy_threshold"]
        self.sell_threshold = params["sell_threshold"]

        self.start(**params)  # Start streaming prices
        
    def on_success(self, data):
        time, symbol, bid, ask = self.parse_tick_data(
            data)
        self.tick_event(time, symbol, bid, ask) 
    
    def tick_event(self, time, symbol, bid, ask):
        midprice = (ask+bid)/2.
        self.prices.loc[time, symbol] = midprice

        resampled_prices = self.prices.resample(
            self.resample_interval,
            how='last',
            fill_method="ffill")

        mean_short = resampled_prices.tail(
            self.mean_period_short).mean()[0]
        mean_long = resampled_prices.tail(
            self.mean_period_long).mean()[0]
        self.beta = mean_short / mean_long

        self.perform_trade_logic(self.beta)
        self.calculate_unrealized_pnl(bid, ask)
        self.print_status()
        
    def perform_trade_logic(self, beta):
        if beta > self.buy_threshold:
            if not self.is_position_opened \
                    or self.position < 0:
                self.check_and_send_order(True)

        elif beta < self.sell_threshold:
            if not self.is_position_opened \
                    or self.position > 0:
                self.check_and_send_order(False)
    def print_status(self):
        print ("[%s] %s pos=%s beta=%s RPnL=%s UPnL=%s" % (
            datetime.now().time(),
            self.instrument, 
            self.position, 
            round(self.beta, 5),
            self.realized_pnl, 
            self.unrealized_pnl)
               )
    def start(self, **params):
        r = pricing.PricingStream(self.accountID,params=params)
        rv = self.api.request(r)
        for tick in rv:
            self.on_success(self, tick)

In [7]:
key = "04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b"
account_id = "101-001-6722193-001"
system = ForexSystem(environment="practice", access_token=key)


In [9]:
key = "04e86c6b0c7bfe71bf7f98d2b1a6496d-dee13f6f97c4508b949c42deaf95925b"
account_id = "101-001-6722193-001"
system = ForexSystem(environment="practice", access_token=key)
system.begin(accountId=account_id,
             instruments="EUR_USD",
             qty=1000,
             resample_interval="10s",
             mean_period_short=5,
             mean_period_long=20,
             buy_threshold=1.0,
             sell_threshold=1.0) 

AttributeError: 'ForexSystem' object has no attribute 'accountID'